In [ ]:
#You WILL need the following in bash:
#    pip install numpy pandas matplotlib seaborn scikit-learn

# MAKE SURE EVERYTHING IS UP TO DATE IF PREDOWNLOADED!!!


In [ ]:
# General things for functions
import numpy as np # Numerical commands
import pandas as pd # Data manipulation

#Visualization tools for plotting
import matplotlib.pyplot as plt 
import seaborn as sns


#Machine learning parts itself
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Importing of the files. Cleaned prior to import.
testAway = pd.read_csv('testDataV3_away.txt')
testHome = pd.read_csv('testDataV3_home.txt')
testY = pd.read_csv('testDataV3_y.txt') # Used to check prediction accuracy. Contains home_win, draw, away_win values

trainAway = pd.read_csv('trainDataV3_away.txt') # Different IDS than test
trainHome = pd.read_csv('trainDataV3_home.txt')
trainY = pd.read_csv('trainDataV3_y.txt') # In training, use this to understand what difference in values leads to a win, draw, loss.

In [ ]:
# Rename columns to distinguish home and away stats
trainAway = trainAway.add_prefix("AWAY_")
trainHome = trainHome.add_prefix("HOME_")

# Ensure features match in the correct order for concatenation
selected_features = ['TEAM_SHOTS_TOTAL_season_sum', 'TEAM_SHOTS_ON_TARGET_season_sum',
                     'TEAM_PASSES_season_sum', 'TEAM_SUCCESSFUL_PASSES_season_sum',
                     'TEAM_SAVES_season_sum', 'TEAM_CORNERS_season_sum',
                     'TEAM_FOULS_season_sum', 'TEAM_YELLOWCARDS_season_sum',
                     'TEAM_REDCARDS_season_sum', 'TEAM_ATTACKS_season_sum',
                     'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_GOALS_season_sum']

# Create full feature lists with correct prefixes
away_features = ["AWAY_" + col for col in selected_features]
home_features = ["HOME_" + col for col in selected_features]

# Concatenate home and away features
X_train = pd.concat([trainAway[away_features], trainHome[home_features]], axis=1)

# Define the target variable
y_train = trainY[['HOME_WINS', 'DRAW', 'AWAY_WINS']]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Initialize a dictionary to store models and their predictions
y_train_pred = {}
y_val_pred = {}

# Train Logistic Regression models
for target in y_train.columns:
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train_scaled, y_train[target])

    # Predict with trainig and validation
    y_train_pred[target] = model.predict(X_train_scaled)
    y_val_pred[target] = model.predict(X_val_scaled)

    train_accuracy = accuracy_score(y_train[target], y_train_pred[target])
    val_accuracy = accuracy_score(y_val[target], y_val_pred[target])

    print(f"Training Accuracy for {target}: {train_accuracy:.4f}")
    print(f"Validation Accuracy for {target}: {val_accuracy:.4f}")  # This should improve

    # Feature importance analysis using Logistic Regression coefficients
    importance = np.abs(model.coef_).flatten()  # Use absolute values for coefficients to measure importance
    features = X_train.columns
    importance_df = pd.DataFrame({'Feature': features, 'Importance': importance})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    # Plot feature importance for home_win/draw/away_win
    plt.figure(figsize=(12, 6))
    sns.barplot(x=importance_df['Feature'], y=importance_df['Importance'])
    plt.xlabel('Feature')
    plt.ylabel('Importance')
    plt.title(f'Feature Importance for {target}')
    plt.xticks(rotation=45, ha='right')  
    plt.tight_layout()
    plt.show()


The below files are to be unused, yet had a previous function prior to better understanding of the desired model.

In [2]:


# def calculate_game_stats(df):
#     # This will calculate the total number of games played
#     df['TOTAL_GAMES_PLAYED'] = df['TEAM_GAME_WON_season_sum'] + df['TEAM_GAME_DRAW_season_sum'] + df['TEAM_GAME_LOST_season_sum']
    
#     # Calculate Win perctengae: Wins / (total of all games)
#     # This is of course to determine "good" compared to "bad" teams and see what factors are different
#     df['WIN_PERCENTAGE'] = (df['TEAM_GAME_WON_season_sum'] / df['TOTAL_GAMES_PLAYED']) *100
    
#     return df


# trainHome = calculate_game_stats(trainHome)
# trainAway = calculate_game_stats(trainAway)
# testHome = calculate_game_stats(testHome)
# testAway = calculate_game_stats(testAway)


# print("Train Home Data with Stats:")
# print(trainHome[['TEAM_GAME_WON_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_LOST_season_sum', 'TOTAL_GAMES_PLAYED', 'WIN_PERCENTAGE']].head())

# print("\nTrain Away Data with Stats:")
# print(trainAway[['TEAM_GAME_WON_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_LOST_season_sum', 'TOTAL_GAMES_PLAYED', 'WIN_PERCENTAGE']].head())


# # Below was a simple test to ensure the test data win loss columns were created
# # This is not to be used until model evaluation with the training data

# # print("\nTest Home Data with Stats:")
# # print(testHome[['TEAM_GAME_WON_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_LOST_season_sum', 'TOTAL_GAMES_PLAYED', 'WIN_LOSS_RATIO']].head())

# # print("\nTest Away Data with Stats:")
# # print(testAway[['TEAM_GAME_WON_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_LOST_season_sum', 'TOTAL_GAMES_PLAYED', 'WIN_LOSS_RATIO']].head())

In [1]:
# def analyze_high_vs_low_winrate(df):
#     # Median win #
#     median_win_percentage = df['WIN_PERCENTAGE'].median()
    
#     # High and low win rate groups (for comparison)
#     high_winrate = df[df['WIN_PERCENTAGE'] >= median_win_percentage]
#     low_winrate = df[df['WIN_PERCENTAGE'] < median_win_percentage]
    
#     # List of statistics to compare
#     stats_to_compare = [
#         'TEAM_SHOTS_TOTAL_season_sum', 'TEAM_SHOTS_ON_TARGET_season_sum',
#         'TEAM_PASSES_season_sum', 'TEAM_SUCCESSFUL_PASSES_season_sum',
#         'TEAM_SAVES_season_sum', 'TEAM_CORNERS_season_sum',
#         'TEAM_FOULS_season_sum', 'TEAM_YELLOWCARDS_season_sum',
#         'TEAM_REDCARDS_season_sum', 'TEAM_ATTACKS_season_sum',
#         'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_GOALS_season_sum'
#     ]
    
#     # Average stats per high/low team
#     high_winrate_stats = high_winrate[stats_to_compare].mean()
#     low_winrate_stats = low_winrate[stats_to_compare].mean()
    
#     # Find the difference to see what matters
#     percentage_diff = ((high_winrate_stats - low_winrate_stats) / low_winrate_stats) * 100
    
#     sorted_diff = percentage_diff.abs().sort_values(ascending=False)

#     print("Top differences between high and low win rate teams:")
#     for stat in sorted_diff.index[:10]:  # Top 10 differences
#         diff = percentage_diff[stat]
#         print(f"{stat}: {diff:.2f}% {'higher' if diff > 0 else 'lower'} for high win rate teams")
    
#     # Shows the top 5 and their average values for comparison
#     top_5_stats = sorted_diff.index[:5]
#     plt.figure(figsize=(12, 6))
#     x = range(len(top_5_stats))
#     plt.bar(x, high_winrate_stats[top_5_stats], width=0.35, align='center', label='High Win Rate')
#     plt.bar(x, low_winrate_stats[top_5_stats], width=0.35, align='edge', label='Low Win Rate')
#     plt.xlabel('Statistics')
#     plt.ylabel('Average Value')
#     plt.title('Top 5 Differences: High vs Low Win Rate Teams')
#     plt.xticks(x, top_5_stats, rotation=45, ha='right')
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

# # Run the analysis for both home and away data
# print("Analysis for Home Teams:")
# analyze_high_vs_low_winrate(trainHome)

# print("\nAnalysis for Away Teams:")
# analyze_high_vs_low_winrate(trainAway)


In [3]:
# # Define important features based above
# important_features = [
#     'TEAM_GOALS_season_sum', 'TEAM_SHOTS_ON_TARGET_season_sum', 
#     'TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_SHOTS_TOTAL_season_sum',
#     'TEAM_PASSES_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_sum',
#     'TEAM_CORNERS_season_sum', 'TEAM_ATTACKS_season_sum', 'WIN_PERCENTAGE'
# ]

# for feature in important_features:
#     testHome[feature] = pd.to_numeric(testHome[feature], errors='coerce')
#     testAway[feature] = pd.to_numeric(testAway[feature], errors='coerce')

# # Function for comparison and therefore prediction
# def predict_match():
#     # User input
#     home_id = int(input("Enter Home Team ID (12303-37670): "))
#     away_id = int(input("Enter Away Team ID (12303-37670): "))

#     # Ensure ID exists
#     if home_id not in testHome['ID'].values or away_id not in testAway['ID'].values:
#         print("Invalid ID entered. Please try again.")
#         return
    
#     # Gets the team's important stats
#     home_team = testHome[testHome['ID'] == home_id].iloc[0]
#     away_team = testAway[testAway['ID'] == away_id].iloc[0]

#     # Find the difference for comparison
#     comparison = {}
#     for feature in important_features:
#         comparison[f'{feature}_DIFFERENCE'] = home_team[feature] - away_team[feature]

#     print("\nStatistical Comparison (Home - Away):")
#     for feature, diff in comparison.items():
#         print(f"{feature}: {diff:.2f}")

#     # Simple rule-based prediction. Could use tweaking but I need help with that
#     if comparison['WIN_PERCENTAGE_DIFFERENCE'] > 0 and \
#        comparison['TEAM_GOALS_season_sum_DIFFERENCE'] > 0 and \
#        comparison['TEAM_SHOTS_ON_TARGET_season_sum_DIFFERENCE'] > 0:
#         predicted_winner = "Home Team"
#     else:
#         predicted_winner = "Away Team"

#     print(f"\nPredicted Winner: {predicted_winner}")

# predict_match()
